# Slicing CDR Relation Extraction 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append('/dfs/scratch0/vschen/metal')

import metal
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
np.set_printoptions(precision=4, suppress=True)

In [4]:
print('PyTorch: ', torch.__version__)
print('MeTaL:   ', metal.__version__)
print('Python:  ', sys.version)
print('Python:  ', sys.version_info)

PyTorch:  0.4.1
MeTaL:    0.3.3
Python:   3.6.7 (default, Dec  8 2018, 17:35:14) 
[GCC 5.4.0 20160609]
Python:   sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


## Initalize CDR Dataset
To uncompress the SQLite db: ```bzip2 -d cdr.db.bz2```

In [5]:
from metal.contrib.backends.wrapper import SnorkelDataset
import os

db_conn_str   = os.path.join(os.getcwd(),"cdr.db")
candidate_def = ['ChemicalDisease', ['chemical', 'disease']]

train, dev, test = SnorkelDataset.splits(db_conn_str, 
                                         candidate_def, 
                                         max_seq_len=125)

print(f'[TRAIN] {len(train)}')
print(f'[DEV]   {len(dev)}')
print(f'[TEST]  {len(test)}')

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
[TRAIN] 8272
[DEV]   888
[TEST]  4620


## Get Pretrained Embeddings

Download [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip):
`wget http://nlp.stanford.edu/data/glove.6B.zip \
&& mkdir -p glove.6B \
&& unzip glove.6B.zip -d glove.6B \
&& rm glove.6B.zip`

In [6]:
from embeddings import EmbeddingLoader, load_embeddings
emb_path  = "../glove.6B/glove.6B.50d.txt"
embs  = EmbeddingLoader(emb_path, fmt='text')

## Generate `L_*` to target slices

In [7]:
from labeling_functions import LFs
print ([lf.__name__ for lf in LFs])

['LF_c_cause_d', 'LF_c_d', 'LF_c_induced_d', 'LF_c_treat_d', 'LF_c_treat_d_wide', 'LF_closer_chem', 'LF_closer_dis', 'LF_ctd_marker_c_d', 'LF_ctd_marker_induce', 'LF_ctd_therapy_treat', 'LF_ctd_unspecified_treat', 'LF_ctd_unspecified_induce', 'LF_d_following_c', 'LF_d_induced_by_c', 'LF_d_induced_by_c_tight', 'LF_d_treat_c', 'LF_develop_d_following_c', 'LF_far_c_d', 'LF_far_d_c', 'LF_improve_before_disease', 'LF_in_ctd_therapy', 'LF_in_ctd_marker', 'LF_in_patient_with', 'LF_induce', 'LF_induce_name', 'LF_induced_other', 'LF_level', 'LF_measure', 'LF_neg_d', 'LF_risk_d', 'LF_treat_d', 'LF_uncertain', 'LF_weak_assertions']


In [8]:
%%time 
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)
L_train = labeler.apply(split=0)
L_dev = labeler.apply(split=1) # used for debugging
L_test = labeler.apply(split=2) # used for evaluation

from snorkel.learning.structure import DependencySelector
ds = DependencySelector()
deps = ds.select(L_train, threshold=0.1)
from snorkel.learning import GenerativeModel

# need to extract `accs` from gen_model
gen_model = GenerativeModel(lf_propensity=True)
gen_model.train(
    L_train, deps=deps, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=0.0
)

accs = np.array(gen_model.learned_lf_stats()['Accuracy'])
accs[np.isnan(accs)] = 0
accs = np.minimum(accs, 0.999)

gen_marginals = gen_model.marginals(L_train)

Clearing existing...


  0%|          | 5/8272 [00:00<02:48, 49.02it/s]

Running UDF...


100%|██████████| 8272/8272 [00:52<00:00, 157.98it/s]


Clearing existing...


...

Clearing existing...
Running UDF...


100%|██████████| 4620/4620 [00:23<00:00, 199.01it/s]


Inferred cardinality: 2


/dfs/scratch0/vschen/snorkel/snorkel/learning/gen_learning.py:350: RuntimeWarning: invalid value encountered in double_scalars
  "Precision": tp / (tp + fp),
/dfs/scratch0/vschen/snorkel/snorkel/learning/gen_learning.py:352: RuntimeWarning: invalid value encountered in double_scalars
  "Accuracy": (tp + tn) / coverage,


CPU times: user 3min 6s, sys: 668 ms, total: 3min 7s
Wall time: 3min 9s


In [9]:
L = L_train.copy()
L[L==-1] = 2 # convert to multiclass
Y_dev = np.array([ex[1] for ex in dev])

In [10]:
from metal.label_model import LabelModel
label_model = LabelModel(k=2, seed=123)
label_model.train_model(L, Y_dev=Y_dev)
label_model.score((L_dev, Y_dev))

Computing O...
Estimating \mu...
[E:0]	Train Loss: 0.492
[E:10]	Train Loss: 0.184
[E:20]	Train Loss: 0.122
[E:30]	Train Loss: 0.076
[E:40]	Train Loss: 0.055
[E:50]	Train Loss: 0.038
[E:60]	Train Loss: 0.033
[E:70]	Train Loss: 0.030
[E:80]	Train Loss: 0.029
[E:90]	Train Loss: 0.028
[E:99]	Train Loss: 0.028
Finished Training
Accuracy: 0.699
        y=1    y=2   
 l=1    81     52    
 l=2    215    540   


0.6993243243243243

### Weak Labels in Dataset

In [11]:
metal_marginals = label_model.predict_proba(L)
metal_marginals

array([[0.1542, 0.8458],
       [1.    , 0.    ],
       [0.2198, 0.7802],
       ...,
       [0.4359, 0.5641],
       [0.3333, 0.6667],
       [0.9703, 0.0297]])

In [12]:
snorkel_marginals = np.vstack((gen_marginals, 1-gen_marginals)).T
snorkel_marginals

array([[0.3856, 0.6144],
       [0.8905, 0.1095],
       [0.6226, 0.3774],
       ...,
       [0.7244, 0.2756],
       [0.5   , 0.5   ],
       [0.7768, 0.2232]])

In [13]:
from metal.contrib.slicing.sqlite_wrapper \
    import SnorkelDataset as SnorkelSliceDataset

train_metal = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    train_marginals=metal_marginals
)

train_snorkel = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    train_marginals=snorkel_marginals
)

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db


### Custom Slicing Dataset

In [14]:
train_slice = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    L_train=L_train.todense()
)

train_slice_metal = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    L_train=L_train.todense(),
    train_marginals=metal_marginals
)

train_slice_snorkel = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    L_train=L_train.todense(),
    train_marginals=snorkel_marginals
)

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db


In [1]:
from metal.contrib.slicing.online_dp import SliceDPModel
from metal.end_model import EndModel
from metal.modules import LSTMModule
def init_model(use_end_model=False, r=None, rw=None):
    wembs = load_embeddings(train.word_dict, embs)
    lstm = LSTMModule(embed_size=50, 
                      hidden_size=100, 
                      embeddings=wembs,
                      lstm_reduction='attention', 
                      dropout=0.0, 
                      num_layers=1, 
                      freeze=False)
    if use_end_model:
        model = EndModel([200, 2], input_module=lstm, seed=123, use_cuda=True)
    else:
        input_layer_config = {
            "input_relu": False,
            "input_batchnorm": False,
            "input_dropout": 0.0,
        }
        model = SliceDPModel(lstm, accs, r, rw, seed=123, use_cuda=True, input_layer_config=input_layer_config)
        
    model.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
    model.config['train_config']['validation_metric'] = 'f1'
    model.config['train_config']['batch_size'] = 32
    model.config['train_config']['n_epochs'] = 10
    return model

from metal.modules import LSTMModule
from metal.tuners import RandomSearchTuner


def search_slice_weights(train_loader, dev_loader, r, rw, max_search=1, search_space=None):
    wembs = load_embeddings(train.word_dict, embs)
    lstm = LSTMModule(embed_size=50, 
                      hidden_size=100, 
                      embeddings=wembs,
                      lstm_reduction='attention', 
                      dropout=0.0, 
                      num_layers=1, 
                      freeze=False)
    
    searcher = RandomSearchTuner(SliceDPModel, validation_metric='f1', log_dir='./run_logs/alpha-loss')

    if search_space is None:
        search_space = {
            "slice_weight": [0, 0.25, 0.5, 0.75, 1.0]
        }

    input_layer_config = {
        "input_relu": False,
        "input_batchnorm": False,
        "input_dropout": 0.0,
    }
    
    trained_model = searcher.search(
        search_space,
        dev_loader,
        train_args=[train_loader],
        init_args=[lstm, accs, r, rw],
        init_kwargs={"use_cuda": True, "input_layer_config": input_layer_config},
        train_kwargs={
            "lr": 0.01,
            "batch_size": 32,
            "n_epochs": 10
        },
        max_search=max_search
    )
    return trained_model

ImportError: No module named 'metal'

## (a) `Oracle`: EndModel Trained on Full GT

In [18]:
oracle = init_model(use_end_model=True)
%time oracle.train_model(train, dev_data=dev)
oracle.score(test, metric=['precision', 'recall', 'f1'])

Process Process-466:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/dfs/scratch0/vschen/snorkel-pytorch/venv/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):


KeyboardInterrupt: 

  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/usr/local/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt


RuntimeError: DataLoader worker (pid 187210) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

## (b) `BaseWeak`: EndModel trained on weak labels

In [ ]:
from metal.end_model import EndModel
from metal.modules import LSTMModule

base_weak = init_model(use_end_model=True)
%time base_weak.train_model(train_snorkel, dev_data=dev)
base_weak_scores = base_weak.score(test, metric=['precision', 'recall', 'f1'])

## (e) `SliceOursWeak`: Slice Model with $\tilde{Y}$ priors

In [17]:
# slice_ours_weak = init_model(r=200, rw=True)
# %time slice_ours_weak.train_model(train_slice_snorkel, dev_data=dev)
%time slice_ours_weak = search_slice_weights(train_slice_snorkel, dev, r=200, rw=True, max_search=20)

slice_ours_weak_scores = slice_ours_weak.score(test, metric=['precision', 'recall', 'f1'])

Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'
Slice Heads:
Reweighting: True
Slice Weight: 5.108583936979527
Input Network: Sequential(
  (0): LSTMModule(
    (embeddings): Embedding(9946, 50)
    (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
  )
)
L_head: Linear(in_features=200, out_features=33, bias=False)
Y_head: Linear(in_features=400, out_features=2, bias=False)
[0] Testing {'slice_weight': 5.108583936979527}
Could not find kwarg "slice_weight" in destination dict.
Using GPU...


Saving model at iteration 0 with best score 0.521
[E:0]	Train Loss: 1.790	Dev f1: 0.521


Saving model at iteration 1 with best score 0.585
[E:1]	Train Loss: 1.754	Dev f1: 0.585


[E:2]	Train Loss: 1.739	Dev f1: 0.568


[E:3]	Train Loss: 1.731	Dev f1: 0.580


...

[E:3]	Train Loss: 2.926	Dev f1: 0.567


[E:4]	Train Loss: 2.927	Dev f1: 0.566


[E:5]	Train Loss: 2.929	Dev f1: 0.569


[E:6]	Train Loss: 2.928	Dev f1: 0.566


[E:7]	Train Loss: 2.927	Dev f1: 0.557


## (f) `SliceUWWeak`: Unweighted Slice model with $\tilde{Y}$ priors

In [ ]:
slice_uw_weak = init_model(r=200, rw=False)
%time slice_uw_weak.train_model(train_slice_snorkel, dev_data=dev)
slice_uw_weak_scores = slice_uw_weak.score(test, metric=['precision', 'recall', 'f1'])

## Slice-specific scores

In [ ]:
# TODO: don't call private fns
Yp_oracle, Y = oracle._get_predictions(test)
Yp_base_weak, Y = base_weak._get_predictions(test)
Yp_slice_ours_weak, Y = slice_ours_weak._get_predictions(test)
Yp_slice_uw_weak, Y = slice_uw_weak._get_predictions(test)

#### `slice_ours` (re-weighting, accuracy priors) vs. `base_weak` (end_model trained on weak labels)

In [ ]:
L_test = L_test.todense()

In [ ]:
from metal.contrib.slicing.experiment_utils import compare_LF_slices

#### `slice_ours_weak` (slice model with weak priors + reweighting) vs. `base_weak` (end_model trained on weak labels)

In [ ]:
print ("slice_ours_weak vs base_weak")
compare_LF_slices(Yp_slice_ours_weak, Yp_base_weak,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.05)

#### `slice_ours_weak` vs. `Yp_slice_uw_weak` (unweighted slice model)

In [ ]:
print ("slice_ours_weak vs. slice_uw_weak")
compare_LF_slices(Yp_slice_ours_weak, Yp_slice_uw_weak,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.05)

In [ ]:
print ("slice_ours_weak vs. oracle")
compare_LF_slices(Yp_slice_ours_weak, Yp_oracle,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.1)

In [ ]:
print ("slice_uw_weak vs. base_weak")
compare_LF_slices(Yp_slice_uw_weak, Yp_base_weak,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.1)